# Chapter 22: SEO & Metadata

Search Engine Optimization (SEO) is crucial for discoverability, social sharing, and providing context to search engines and crawlers. Next.js 13+ introduces a powerful Metadata API that allows you to declaratively define metadata for each route, generate dynamic SEO content based on data, and automatically handle complex requirements like Open Graph images and structured data.

By the end of this chapter, you'll master the Metadata API for static and dynamic meta tags, implement Open Graph and Twitter Card optimization, inject JSON-LD structured data for rich snippets, generate dynamic sitemaps and robots.txt files, manage canonical URLs for duplicate content prevention, and optimize social media previews across all platforms.

## 22.1 Metadata API

The Metadata API provides a declarative way to define document head metadata using static exports or dynamic generation.

### Static Metadata

Define metadata in page or layout files:

```typescript
// app/page.tsx
import type { Metadata } from 'next';

// Static metadata for the home page
export const metadata: Metadata = {
  title: 'Next.js Mastery - Build Modern Web Applications',
  description: 'Learn Next.js with comprehensive tutorials, examples, and best practices for building production-ready applications.',
  
  // Keywords (less important for Google but used by others)
  keywords: ['Next.js', 'React', 'Tutorial', 'Web Development'],
  
  // Authors and creators
  authors: [{ name: 'John Doe', url: 'https://twitter.com/johndoe' }],
  creator: 'Next.js Mastery Team',
  publisher: 'Vercel',
  
  // Robots directive
  robots: {
    index: true,
    follow: true,
    nocache: false,
    googleBot: {
      index: true,
      follow: true,
      noimageindex: false,
      'max-video-preview': -1,
      'max-image-preview': 'large',
      'max-snippet': -1,
    },
  },
  
  // Viewport and theme
  viewport: {
    width: 'device-width',
    initialScale: 1,
    maximumScale: 5,
    themeColor: [
      { media: '(prefers-color-scheme: light)', color: 'white' },
      { media: '(prefers-color-scheme: dark)', color: 'black' },
    ],
  },
  
  // Icons
  icons: {
    icon: '/favicon.ico',
    shortcut: '/favicon-16x16.png',
    apple: '/apple-touch-icon.png',
    other: [
      {
        rel: 'apple-touch-icon-precomposed',
        url: '/apple-touch-icon-precomposed.png',
      },
    ],
  },
  
  // Manifest for PWA
  manifest: '/manifest.json',
  
  // Twitter-specific
  twitter: {
    card: 'summary_large_image',
    title: 'Next.js Mastery',
    description: 'Build modern web applications with Next.js',
    siteId: '1234567890',
    creator: '@nextjs',
    creatorId: '1234567890',
    images: ['https://example.com/og-image.jpg'],
  },
  
  // Open Graph
  openGraph: {
    title: 'Next.js Mastery',
    description: 'Build modern web applications',
    url: 'https://example.com',
    siteName: 'Next.js Mastery',
    images: [
      {
        url: 'https://example.com/og-image.jpg',
        width: 1200,
        height: 630,
        alt: 'Next.js Mastery Banner',
      },
    ],
    locale: 'en_US',
    type: 'website',
  },
  
  // Verification for search consoles
  verification: {
    google: 'google-site-verification-code',
    yandex: 'yandex-verification-code',
    yahoo: 'yahoo-verification-code',
    other: {
      me: ['my-email@example.com', 'my-link@example.com'],
    },
  },
  
  // Alternate languages/regions
  alternates: {
    canonical: 'https://example.com',
    languages: {
      'en-US': 'https://example.com/en',
      'de-DE': 'https://example.com/de',
      'fr-FR': 'https://example.com/fr',
    },
    types: {
      'application/rss+xml': [
        { url: 'feed.xml', title: 'RSS Feed' },
        { url: 'feed.atom', title: 'Atom Feed' },
      ],
    },
  },
  
  // Category and classification
  category: 'technology',
  classification: 'web development',
  
  // Other metadata
  referrer: 'origin-when-cross-origin',
  colorScheme: 'dark light',
  formatDetection: {
    telephone: false,
    date: false,
    address: false,
    email: false,
  },
};

export default function HomePage() {
  return <div>Home</div>;
}
```

### Layout Metadata Inheritance

Metadata in layouts cascades to child pages:

```typescript
// app/layout.tsx (Root layout)
import type { Metadata } from 'next';

export const metadata: Metadata = {
  title: {
    template: '%s | Next.js Mastery',
    default: 'Next.js Mastery - Build Modern Web Applications',
  },
  description: 'The best place to learn Next.js',
  openGraph: {
    type: 'website',
    locale: 'en_US',
    url: 'https://example.com',
    siteName: 'Next.js Mastery',
  },
  twitter: {
    card: 'summary_large_image',
  },
};

// app/blog/page.tsx (Child page)
import type { Metadata } from 'next';

export const metadata: Metadata = {
  title: 'Blog', // Results in: "Blog | Next.js Mastery"
  description: 'Read our latest articles about Next.js and React',
};

// app/blog/[slug]/page.tsx (Dynamic page)
export async function generateMetadata({ params }): Promise<Metadata> {
  return {
    title: post.title, // Results in: "Post Title | Next.js Mastery"
    description: post.excerpt,
  };
}
```

## 22.2 Dynamic Metadata

Generate metadata based on route parameters or fetched data.

### Data-Driven Metadata

Fetch data to generate accurate SEO tags:

```typescript
// app/blog/[slug]/page.tsx
import type { Metadata, ResolvingMetadata } from 'next';
import { notFound } from 'next/navigation';

interface PageProps {
  params: { slug: string };
}

// Generate metadata based on post data
export async function generateMetadata(
  { params }: PageProps,
  parent: ResolvingMetadata
): Promise<Metadata> {
  // Fetch post data
  const post = await getPost(params.slug);
  
  if (!post) {
    return {
      title: 'Post Not Found',
    };
  }
  
  // Access parent metadata (from layout)
  const previousImages = (await parent).openGraph?.images || [];
  
  return {
    title: post.title,
    description: post.excerpt,
    authors: [{ name: post.author.name }],
    openGraph: {
      title: post.title,
      description: post.excerpt,
      type: 'article',
      publishedTime: post.publishedAt,
      modifiedTime: post.updatedAt,
      authors: [post.author.name],
      tags: post.tags,
      images: [
        {
          url: post.ogImage || post.coverImage,
          width: 1200,
          height: 630,
          alt: post.title,
        },
        ...previousImages, // Include parent images as fallback
      ],
    },
    twitter: {
      card: 'summary_large_image',
      title: post.title,
      description: post.excerpt,
      images: [post.ogImage || post.coverImage],
    },
    alternates: {
      canonical: `/blog/${post.slug}`,
    },
    robots: {
      index: post.published,
      follow: post.published,
      nocache: !post.published,
    },
  };
}

export default async function BlogPost({ params }: PageProps) {
  const post = await getPost(params.slug);
  
  if (!post) notFound();
  
  return (
    <article>
      <h1>{post.title}</h1>
      <div>{post.content}</div>
    </article>
  );
}
```

### E-commerce Product Metadata

Optimize product pages for shopping search results:

```typescript
// app/shop/[category]/[product]/page.tsx
export async function generateMetadata({
  params,
}: {
  params: { category: string; product: string };
}): Promise<Metadata> {
  const product = await getProduct(params.product);
  
  if (!product) return { title: 'Product Not Found' };
  
  return {
    title: `${product.name} - Buy Now | Shop`,
    description: product.metaDescription || product.description.slice(0, 160),
    openGraph: {
      title: product.name,
      description: product.shortDescription,
      type: 'product',
      url: `/shop/${params.category}/${params.product}`,
      images: product.images.map((img, idx) => ({
        url: img.url,
        width: 1200,
        height: 630,
        alt: idx === 0 ? product.name : `${product.name} - View ${idx + 1}`,
      })),
    },
    twitter: {
      card: 'summary_large_image',
      title: product.name,
      description: product.shortDescription,
      images: [product.images[0]?.url],
    },
    alternates: {
      canonical: `/shop/${params.category}/${params.product}`,
      languages: Object.fromEntries(
        product.translations.map((t) => [
          t.locale,
          `/shop/${params.category}/${params.product}?lang=${t.locale}`,
        ])
      ),
    },
    robots: {
      index: product.inStock,
      follow: true,
    },
  };
}
```

## 22.3 Open Graph Tags

Open Graph protocol enables rich sharing previews on Facebook, LinkedIn, Discord, and other platforms.

### Dynamic OG Image Generation

Generate custom Open Graph images dynamically:

```typescript
// app/api/og/route.tsx
import { ImageResponse } from 'next/og';
import { NextRequest } from 'next/server';

export const runtime = 'edge';

export async function GET(request: NextRequest) {
  const { searchParams } = request.nextUrl;
  const title = searchParams.get('title') || 'Next.js Mastery';
  const description = searchParams.get('description') || 'Build modern web apps';
  
  return new ImageResponse(
    (
      <div
        style={{
          height: '100%',
          width: '100%',
          display: 'flex',
          flexDirection: 'column',
          alignItems: 'center',
          justifyContent: 'center',
          backgroundColor: 'white',
          backgroundImage: 'linear-gradient(to bottom right, #667eea, #764ba2)',
        }}
      >
        <div
          style={{
            display: 'flex',
            flexDirection: 'column',
            alignItems: 'center',
            justifyContent: 'center',
            backgroundColor: 'rgba(255, 255, 255, 0.95)',
            padding: '60px 80px',
            borderRadius: '20px',
            boxShadow: '0 20px 60px rgba(0,0,0,0.3)',
          }}
        >
          <h1
            style={{
              fontSize: '60px',
              fontWeight: 'bold',
              color: '#1a202c',
              textAlign: 'center',
              lineHeight: 1.2,
              marginBottom: '20px',
            }}
          >
            {title}
          </h1>
          <p
            style={{
              fontSize: '32px',
              color: '#4a5568',
              textAlign: 'center',
              maxWidth: '800px',
            }}
          >
            {description}
          </p>
          <div
            style={{
              marginTop: '40px',
              fontSize: '24px',
              color: '#667eea',
              fontWeight: '600',
            }}
          >
            nextjs-mastery.com
          </div>
        </div>
      </div>
    ),
    {
      width: 1200,
      height: 630,
      headers: {
        'Cache-Control': 'public, max-age=86400, stale-while-revalidate=604800',
      },
    }
  );
}

// Usage in metadata:
// openGraph: {
//   images: [`/api/og?title=${encodeURIComponent(title)}&description=${encodeURIComponent(description)}`],
// }
```

### Structured OG Data

Complete Open Graph implementation for different content types:

```typescript
// lib/og-metadata.ts
import { Metadata } from 'next';

export function generateArticleOG(
  post: {
    title: string;
    excerpt: string;
    slug: string;
    publishedAt: string;
    updatedAt?: string;
    author: { name: string; image?: string };
    coverImage: string;
    tags: string[];
  }
): Metadata['openGraph'] {
  return {
    type: 'article',
    title: post.title,
    description: post.excerpt,
    url: `https://example.com/blog/${post.slug}`,
    publishedTime: post.publishedAt,
    modifiedTime: post.updatedAt,
    authors: [post.author.name],
    tags: post.tags,
    section: 'Technology',
    images: [
      {
        url: post.coverImage,
        width: 1200,
        height: 630,
        alt: post.title,
      },
    ],
  };
}

export function generateProductOG(product: {
  name: string;
  description: string;
  price: number;
  currency: string;
  image: string;
  availability: 'in stock' | 'out of stock';
}): Metadata['openGraph'] {
  return {
    type: 'product',
    title: product.name,
    description: product.description,
    url: `https://example.com/shop/${product.slug}`,
    images: [
      {
        url: product.image,
        width: 1200,
        height: 630,
        alt: product.name,
      },
    ],
  };
}

// Usage:
// export async function generateMetadata({ params }) {
//   const post = await getPost(params.slug);
//   return {
//     openGraph: generateArticleOG(post),
//   };
// }
```

## 22.4 Structured Data (JSON-LD)

JSON-LD helps search engines understand your content and display rich snippets.

### Article Schema

Mark up blog posts for rich results:

```typescript
// components/json-ld/article.tsx
import Script from 'next/script';

interface ArticleJsonLdProps {
  title: string;
  description: string;
  slug: string;
  publishedAt: string;
  updatedAt?: string;
  author: {
    name: string;
    url?: string;
    image?: string;
  };
  coverImage: string;
  tags: string[];
}

export function ArticleJsonLd({
  title,
  description,
  slug,
  publishedAt,
  updatedAt,
  author,
  coverImage,
  tags,
}: ArticleJsonLdProps) {
  const jsonLd = {
    '@context': 'https://schema.org',
    '@type': 'TechArticle',
    headline: title,
    description: description,
    image: coverImage,
    datePublished: publishedAt,
    dateModified: updatedAt || publishedAt,
    author: {
      '@type': 'Person',
      name: author.name,
      url: author.url,
      image: author.image,
    },
    publisher: {
      '@type': 'Organization',
      name: 'Next.js Mastery',
      logo: {
        '@type': 'ImageObject',
        url: 'https://example.com/logo.png',
      },
    },
    url: `https://example.com/blog/${slug}`,
    mainEntityOfPage: {
      '@type': 'WebPage',
      '@id': `https://example.com/blog/${slug}`,
    },
    keywords: tags.join(', '),
    inLanguage: 'en-US',
  };

  return (
    <Script
      id="article-jsonld"
      type="application/ld+json"
      dangerouslySetInnerHTML={{ __html: JSON.stringify(jsonLd) }}
    />
  );
}

// Usage in page:
// export default async function Page({ params }) {
//   const post = await getPost(params.slug);
//   return (
//     <>
//       <ArticleJsonLd {...post} />
//       <article>{post.content}</article>
//     </>
//   );
// }
```

### Product Schema

E-commerce structured data for shopping results:

```typescript
// components/json-ld/product.tsx
interface ProductJsonLdProps {
  name: string;
  description: string;
  image: string;
  sku: string;
  brand: string;
  price: number;
  currency: string;
  availability: 'InStock' | 'OutOfStock' | 'PreOrder';
  rating?: {
    value: number;
    count: number;
  };
}

export function ProductJsonLd({
  name,
  description,
  image,
  sku,
  brand,
  price,
  currency,
  availability,
  rating,
}: ProductJsonLdProps) {
  const jsonLd = {
    '@context': 'https://schema.org',
    '@type': 'Product',
    name,
    description,
    image,
    sku,
    brand: {
      '@type': 'Brand',
      name: brand,
    },
    offers: {
      '@type': 'Offer',
      url: typeof window !== 'undefined' ? window.location.href : '',
      priceCurrency: currency,
      price: price.toString(),
      availability: `https://schema.org/${availability}`,
      itemCondition: 'https://schema.org/NewCondition',
    },
    ...(rating && {
      aggregateRating: {
        '@type': 'AggregateRating',
        ratingValue: rating.value,
        reviewCount: rating.count,
      },
    }),
  };

  return (
    <Script
      id="product-jsonld"
      type="application/ld+json"
      dangerouslySetInnerHTML={{ __html: JSON.stringify(jsonLd) }}
    />
  );
}
```

### Organization & Website Schema

Global structured data for brand recognition:

```typescript
// app/layout.tsx
import Script from 'next/script';

export default function RootLayout({ children }: { children: React.ReactNode }) {
  const websiteJsonLd = {
    '@context': 'https://schema.org',
    '@type': 'WebSite',
    name: 'Next.js Mastery',
    url: 'https://example.com',
    potentialAction: {
      '@type': 'SearchAction',
      target: {
        '@type': 'EntryPoint',
        urlTemplate: 'https://example.com/search?q={search_term_string}',
      },
      'query-input': 'required name=search_term_string',
    },
  };

  const organizationJsonLd = {
    '@context': 'https://schema.org',
    '@type': 'Organization',
    name: 'Next.js Mastery',
    url: 'https://example.com',
    logo: 'https://example.com/logo.png',
    sameAs: [
      'https://twitter.com/nextjs',
      'https://github.com/vercel/next.js',
      'https://linkedin.com/company/nextjs',
    ],
    contactPoint: {
      '@type': 'ContactPoint',
      telephone: '+1-123-456-7890',
      contactType: 'customer service',
      availableLanguage: ['English'],
    },
  };

  return (
    <html lang="en">
      <head>
        <Script
          id="website-jsonld"
          type="application/ld+json"
          dangerouslySetInnerHTML={{ __html: JSON.stringify(websiteJsonLd) }}
        />
        <Script
          id="organization-jsonld"
          type="application/ld+json"
          dangerouslySetInnerHTML={{ __html: JSON.stringify(organizationJsonLd) }}
        />
      </head>
      <body>{children}</body>
    </html>
  );
}
```

## 22.5 Sitemap Generation

Sitemaps help search engines discover and index your pages efficiently.

### Static Sitemap

Simple sitemap for static routes:

```typescript
// app/sitemap.ts
import { MetadataRoute } from 'next';

export default function sitemap(): MetadataRoute.Sitemap {
  return [
    {
      url: 'https://example.com',
      lastModified: new Date(),
      changeFrequency: 'yearly',
      priority: 1,
    },
    {
      url: 'https://example.com/about',
      lastModified: new Date(),
      changeFrequency: 'monthly',
      priority: 0.8,
    },
    {
      url: 'https://example.com/blog',
      lastModified: new Date(),
      changeFrequency: 'weekly',
      priority: 0.9,
    },
    {
      url: 'https://example.com/contact',
      lastModified: new Date(),
      changeFrequency: 'yearly',
      priority: 0.5,
    },
  ];
}
```

### Dynamic Sitemap

Generate sitemaps from database content:

```typescript
// app/sitemap.ts
import { MetadataRoute } from 'next';
import { db } from '@/lib/db';

export default async function sitemap(): Promise<MetadataRoute.Sitemap> {
  // Fetch all published posts
  const posts = await db.post.findMany({
    where: { published: true },
    select: { slug: true, updatedAt: true },
  });
  
  // Fetch all products
  const products = await db.product.findMany({
    where: { isActive: true },
    select: { slug: true, updatedAt: true, category: { select: { slug: true } } },
  });
  
  // Base routes
  const routes = [
    {
      url: 'https://example.com',
      lastModified: new Date(),
      changeFrequency: 'daily' as const,
      priority: 1,
    },
    {
      url: 'https://example.com/blog',
      lastModified: new Date(),
      changeFrequency: 'daily' as const,
      priority: 0.9,
    },
    {
      url: 'https://example.com/shop',
      lastModified: new Date(),
      changeFrequency: 'daily' as const,
      priority: 0.9,
    },
  ];
  
  // Add post routes
  const postRoutes = posts.map((post) => ({
    url: `https://example.com/blog/${post.slug}`,
    lastModified: post.updatedAt,
    changeFrequency: 'weekly' as const,
    priority: 0.8,
  }));
  
  // Add product routes
  const productRoutes = products.map((product) => ({
    url: `https://example.com/shop/${product.category.slug}/${product.slug}`,
    lastModified: product.updatedAt,
    changeFrequency: 'weekly' as const,
    priority: 0.7,
  }));
  
  return [...routes, ...postRoutes, ...productRoutes];
}
```

### Multi-Sitemap Architecture

For large sites, split sitemaps:

```typescript
// app/sitemap.xml/route.ts
import { getServerSideSitemap } from 'next-sitemap';

// For sites with > 50,000 URLs, split into multiple sitemaps
export async function GET(request: Request) {
  // Logic to generate index sitemap or specific chunk
  return getServerSideSitemap([
    {
      loc: 'https://example.com/sitemap-pages.xml',
      lastmod: new Date().toISOString(),
    },
    {
      loc: 'https://example.com/sitemap-posts.xml',
      lastmod: new Date().toISOString(),
    },
    {
      loc: 'https://example.com/sitemap-products.xml',
      lastmod: new Date().toISOString(),
    },
  ]);
}

// app/sitemap-posts.xml/route.ts
export async function GET() {
  const posts = await db.post.findMany();
  
  return getServerSideSitemap(
    posts.map((post) => ({
      loc: `https://example.com/blog/${post.slug}`,
      lastmod: post.updatedAt.toISOString(),
      changefreq: 'weekly',
      priority: 0.8,
    }))
  );
}
```

## 22.6 Robots.txt Configuration

Control crawler access to your site.

### Static Robots.txt

Simple configuration file:

```typescript
// app/robots.ts
import { MetadataRoute } from 'next';

export default function robots(): MetadataRoute.Robots {
  return {
    rules: [
      {
        userAgent: '*',
        allow: '/',
        disallow: ['/private/', '/admin/', '/api/', '/_next/'],
      },
      {
        userAgent: 'Googlebot',
        allow: '/',
        disallow: ['/nogooglebot/'],
      },
      {
        userAgent: 'Googlebot-Image',
        disallow: '/private-images/',
      },
    ],
    sitemap: 'https://example.com/sitemap.xml',
    host: 'https://example.com',
  };
}
```

### Dynamic Robots.txt

Generate based on environment or user settings:

```typescript
// app/robots.ts
import { MetadataRoute } from 'next';

export default function robots(): MetadataRoute.Robots {
  const isProduction = process.env.NODE_ENV === 'production';
  
  if (!isProduction) {
    // Disallow all crawlers in development/staging
    return {
      rules: {
        userAgent: '*',
        disallow: '/',
      },
    };
  }
  
  return {
    rules: [
      {
        userAgent: '*',
        allow: '/',
        disallow: [
          '/api/',
          '/admin/',
          '/private/',
          '/auth/',
          '/*.json$',  // Disallow JSON endpoints
          '/search?',  // Disallow search result pages
        ],
        crawlDelay: 10,  // Be nice to servers
      },
    ],
    sitemap: 'https://example.com/sitemap.xml',
    host: 'https://example.com',
  };
}
```

## 22.7 Canonical URLs

Prevent duplicate content issues with proper canonicalization.

### Handling Query Parameters

Manage URL variations for SEO:

```typescript
// app/products/page.tsx
import type { Metadata } from 'next';

interface PageProps {
  searchParams: { [key: string]: string | string[] | undefined };
}

export function generateMetadata({
  searchParams,
}: PageProps): Metadata {
  // Build canonical URL without tracking parameters
  const cleanParams = new URLSearchParams();
  
  // Only include SEO-relevant parameters
  if (searchParams.category) {
    cleanParams.set('category', searchParams.category.toString());
  }
  if (searchParams.sort) {
    cleanParams.set('sort', searchParams.sort.toString());
  }
  
  const queryString = cleanParams.toString();
  const canonical = queryString 
    ? `https://example.com/products?${queryString}`
    : 'https://example.com/products';
  
  return {
    title: 'Products',
    alternates: {
      canonical,
      // Indicate language variations
      languages: {
        'en-US': `https://example.com/products${queryString ? `?${queryString}` : ''}`,
        'es-ES': `https://example.com/es/products${queryString ? `?${queryString}` : ''}`,
      },
    },
    // Prevent indexing of filtered pages with no results or pure pagination
    robots: {
      index: !searchParams.page || searchParams.page === '1',
      follow: true,
    },
  };
}
```

### Pagination Canonicalization

Properly handle paginated content:

```typescript
// app/blog/page.tsx
interface PageProps {
  searchParams: { page?: string };
}

export function generateMetadata({
  searchParams,
}: PageProps): Metadata {
  const currentPage = parseInt(searchParams.page || '1');
  const isFirstPage = currentPage === 1;
  
  return {
    title: isFirstPage ? 'Blog' : `Blog - Page ${currentPage}`,
    alternates: {
      // Always point to first page as canonical (or self-referential per strategy)
      canonical: 'https://example.com/blog',
      // Pagination with prev/next
      prev: currentPage > 1 ? `https://example.com/blog?page=${currentPage - 1}` : undefined,
      next: `https://example.com/blog?page=${currentPage + 1}`,
    },
    // Only index first page to avoid duplicate content
    robots: {
      index: isFirstPage,
      follow: true,
    },
  };
}
```

## 22.8 Social Media Optimization

Optimize how content appears when shared on social platforms.

### Twitter Cards

Detailed Twitter optimization:

```typescript
// lib/twitter-metadata.ts
import { Metadata } from 'next';

export function generateTwitterCard(
  type: 'summary' | 'summary_large_image' | 'app' | 'player',
  data: {
    title: string;
    description: string;
    image?: string;
    creator?: string;
    site?: string;
  }
): Metadata['twitter'] {
  const base = {
    card: type,
    title: data.title,
    description: data.description,
    creator: data.creator || '@defaulthandle',
    site: data.site || '@sitehandle',
  };
  
  if (data.image && type !== 'app') {
    return {
      ...base,
      images: [data.image],
    };
  }
  
  return base;
}

// Special handling for video content
export function generateTwitterPlayerCard(video: {
  title: string;
  description: string;
  playerUrl: string;
  width: number;
  height: number;
  image: string;
}): Metadata['twitter'] {
  return {
    card: 'player',
    title: video.title,
    description: video.description,
    players: {
      playerUrl: video.playerUrl,
      streamUrl: video.playerUrl,
      width: video.width,
      height: video.height,
    },
    images: [video.image],
  };
}
```

### Social Preview Testing

Implement meta tags for link preview debugging:

```typescript
// app/debug/og-preview/page.tsx
export const metadata: Metadata = {
  title: 'OG Preview Debug',
  robots: { index: false }, // Don't index debug page
};

export default function OGPreviewPage() {
  return (
    <div className="p-8 max-w-4xl mx-auto">
      <h1 className="text-2xl font-bold mb-4">Social Preview Debugger</h1>
      
      <div className="space-y-8">
        <section>
          <h2 className="text-xl font-semibold mb-2">Open Graph</h2>
          <div className="bg-gray-100 p-4 rounded">
            <p>Check your Open Graph tags using:</p>
            <ul className="list-disc pl-5 mt-2 space-y-1">
              <li>
                <a 
                  href="https://developers.facebook.com/tools/debug/" 
                  className="text-blue-600 hover:underline"
                  target="_blank"
                  rel="noopener noreferrer"
                >
                  Facebook Sharing Debugger
                </a>
              </li>
              <li>
                <a 
                  href="https://www.linkedin.com/post-inspector/" 
                  className="text-blue-600 hover:underline"
                  target="_blank"
                  rel="noopener noreferrer"
                >
                  LinkedIn Post Inspector
                </a>
              </li>
            </ul>
          </div>
        </section>
        
        <section>
          <h2 className="text-xl font-semibold mb-2">Twitter Cards</h2>
          <div className="bg-gray-100 p-4 rounded">
            <a 
              href="https://cards-dev.twitter.com/validator" 
              className="text-blue-600 hover:underline"
              target="_blank"
              rel="noopener noreferrer"
            >
              Twitter Card Validator
            </a>
          </div>
        </section>
      </div>
    </div>
  );
}
```

## Key Takeaways from Chapter 22

1. **Metadata API**: Use the `Metadata` type from `next` to define static metadata in page/layout files. For dynamic content, implement `generateMetadata()` async functions that can fetch data and return metadata objects. Metadata cascades from layouts to pages, with child metadata merging with or overriding parent metadata.

2. **Dynamic SEO**: Generate title, description, OG images, and canonical URLs based on database content in `generateMetadata()`. Access parent metadata using the `parent` parameter to build upon layout defaults. Use conditional logic to set `robots: { index: false }` for unpublished or private content.

3. **Open Graph Optimization**: Implement dynamic OG image generation using the Edge Runtime and `ImageResponse` for custom social sharing cards. Specify OG type (`website`, `article`, `product`) with appropriate properties (publishedTime, authors, prices) for rich previews on Facebook, LinkedIn, and Discord.

4. **Structured Data**: Inject JSON-LD using Next.js `Script` component with `type="application/ld+json"` to enable rich snippets in search results. Implement Schema.org types like `Article`, `Product`, `Organization`, and `WebSite` to help search engines understand content context and display enhanced results.

5. **Sitemap Generation**: Export a default function from `app/sitemap.ts` that returns an array of URL objects with `url`, `lastModified`, `changeFrequency`, and `priority`. For large sites, split sitemaps by content type (pages, posts, products) and reference them in a sitemap index.

6. **Robots.txt Management**: Generate `robots.txt` dynamically from `app/robots.ts` to control crawler access differently per environment (disallow all in staging). Specify sitemap location and host, and use `crawlDelay` for rate limiting.

7. **Canonical URLs & Pagination**: Set canonical URLs in `alternates.canonical` to consolidate duplicate content (filter variations, tracking parameters). Use `alternates.prev` and `alternates.next` for pagination series, and consider `robots: { index: false }` for paginated pages beyond page 1 to prevent thin content issues.

## Coming Up Next

**Chapter 23: Security Best Practices**

Now that your application is optimized for search and social sharing, it's critical to protect it from threats. In Chapter 23, we'll explore common security vulnerabilities in Next.js applications, environment variable management, Content Security Policy (CSP) implementation, XSS and CSRF prevention, secure authentication patterns, dependency security scanning, and data protection strategies. You'll learn how to build applications that are not only fast and discoverable but also resilient against attacks.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='21. performance_optimization.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='23. security_best_practices.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
